In [1]:
import pandas as pd
import numpy as np
import bs4 as bs
import urllib.request

In [2]:
link = "https://en.wikipedia.org/wiki/List_of_American_films_of_2018"

In [3]:
source = urllib.request.urlopen(link).read()
soup = bs.BeautifulSoup(source,'lxml')

In [4]:
tables = soup.find_all('table',class_='wikitable sortable')

In [5]:
len(tables)

4

In [6]:
df1 = pd.read_html(str(tables[0]))[0]
df2 = pd.read_html(str(tables[1]))[0]
df3 = pd.read_html(str(tables[2]))[0]
df4 = pd.read_html(str(tables[3]).replace("'1\"\'",'"1"'))

In [7]:
df = df1.append(df2.append(df3.append(df4,ignore_index=True),ignore_index=True),ignore_index=True)

In [8]:
df

,Opening,Opening.1,Title,Production company,Cast and crew,.mw-parser-output .tooltip-dotted{border-bottom:1px dotted;cursor:help}Ref.,Ref.
0,JANUARY,5,Insidious: The Last Key,Universal Pictures / Blumhouse Productions / S...,Adam Robitel (director); Leigh Whannell (scree...,[2],NaN
1,JANUARY,5,The Strange Ones,Vertical Entertainment,Lauren Wolkstein (director); Christopher Radcl...,[3],NaN
2,JANUARY,5,Stratton,Momentum Pictures,"Simon West (director); Duncan Falconer, Warren...",[4],NaN
3,JANUARY,10,Sweet Country,Samuel Goldwyn Films,"Warwick Thornton (director); David Tranter, St...",[5],NaN
4,JANUARY,12,The Commuter,Lionsgate / StudioCanal / The Picture Company,Jaume Collet-Serra (director); Byron Willinger...,[6],NaN
...,...,...,...,...,...,...,...
269,DECEMBER,25,Holmes & Watson,Columbia Pictures / Gary Sanchez Productions,Etan Cohen (director/screenplay); Will Ferrell...,NaN,[164]
270,DECEMBER,25,Vice,Annapurna Pictures / Plan B Entertainment,Adam McKay (director/screenplay); Christian Ba...,NaN,[137]
271,DECEMBER,25,On the Basis of Sex,Focus Features,Mimi Leder (director); Daniel Stiepleman (scre...,NaN,[229]
272,DECEMBER,25,Destroyer,Annapurna Pictures,"Karyn Kusama (director); Phil Hay, Matt Manfre...",NaN,[262]


In [9]:
df_2018 = df[['Title']]

In [10]:
df_2018

,Title
0,Insidious: The Last Key
1,The Strange Ones
2,Stratton
3,Sweet Country
4,The Commuter
...,...
269,Holmes & Watson
270,Vice
271,On the Basis of Sex
272,Destroyer


In [11]:
!pip install tmdbv3api

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [12]:
from tmdbv3api import TMDb
import json
tmdb = TMDb()
tmdb.api_key = '2d71ac9f8f76e2fe996cdc92cab9f9cd'

In [13]:
from tmdbv3api import Movie
import json
tmdb_movie = Movie() 

In [14]:
def ids(x):
  result = tmdb_movie.search(x)
  if not result:
    return np.NaN
  else:
    return int(result[0].id)

In [15]:
df_2018['id'] = df_2018['Title'].map(lambda x: ids(str(x)))

<ipython-input-15-b080d919ec1a>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2018['id'] = df_2018['Title'].map(lambda x: ids(str(x)))


In [16]:
df_2018

,Title,id
0,Insidious: The Last Key,406563
1,The Strange Ones,426258
2,Stratton,348389
3,Sweet Country,468210
4,The Commuter,399035
...,...,...
269,Holmes & Watson,426563
270,Vice,82
271,On the Basis of Sex,339380
272,Destroyer,1040330


In [17]:
df_2018[df_2018['id'].isnull()]

,Title,id


In [18]:
df_2018['id'] = df_2018['id'].astype('int')

<ipython-input-18-e57be94d266a>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2018['id'] = df_2018['id'].astype('int')


In [19]:
df_2018

,Title,id
0,Insidious: The Last Key,406563
1,The Strange Ones,426258
2,Stratton,348389
3,Sweet Country,468210
4,The Commuter,399035
...,...,...
269,Holmes & Watson,426563
270,Vice,82
271,On the Basis of Sex,339380
272,Destroyer,1040330


In [20]:
def description(movie_id):
        description =""
    
        movie_details = tmdb_movie.details(movie_id)
        if movie_details['overview'] and movie_details['tagline']:
          return movie_details['overview'] + "@#$" + movie_details['tagline']
        elif movie_details['overview'] and not movie_details['tagline']:
          return movie_details['overview'] + "@#$" + ""
        elif not movie_details['overview'] and movie_details['tagline']:
          return ""+ "@#$" + movie_details['tagline']
        else:
          return "@#$"

In [21]:
df_2018['description'] = df_2018['id'].map(lambda x: description(int(x)))

<ipython-input-21-14c045c10226>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2018['description'] = df_2018['id'].map(lambda x: description(int(x)))


In [22]:
df_2018

,Title,id,description
0,Insidious: The Last Key,406563,Parapsychologist Elise Rainier and her team tr...
1,The Strange Ones,426258,Mysterious events surround the travels of two ...
2,Stratton,348389,A British Special Boat Service commando tracks...
3,Sweet Country,468210,In 1929 in the outback of the Northern Territo...
4,The Commuter,399035,"A businessman, on his daily commute home, gets..."
...,...,...,...
269,Holmes & Watson,426563,Detective Sherlock Holmes and Dr. John Watson ...
270,Vice,82,A case involving drug lords and murder in Sout...
271,On the Basis of Sex,339380,Young lawyer Ruth Bader Ginsburg teams with he...
272,Destroyer,1040330,Ahead of the release of upcoming fantasy film ...


In [23]:
df_2018['description'] = df_2018['description'].apply(lambda x: x.split("@#$"))

<ipython-input-23-28270633798a>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2018['description'] = df_2018['description'].apply(lambda x: x.split("@#$"))


In [24]:
df_2018

,Title,id,description
0,Insidious: The Last Key,406563,[Parapsychologist Elise Rainier and her team t...
1,The Strange Ones,426258,[Mysterious events surround the travels of two...
2,Stratton,348389,[A British Special Boat Service commando track...
3,Sweet Country,468210,[In 1929 in the outback of the Northern Territ...
4,The Commuter,399035,"[A businessman, on his daily commute home, get..."
...,...,...,...
269,Holmes & Watson,426563,[Detective Sherlock Holmes and Dr. John Watson...
270,Vice,82,[A case involving drug lords and murder in Sou...
271,On the Basis of Sex,339380,[Young lawyer Ruth Bader Ginsburg teams with h...
272,Destroyer,1040330,[Ahead of the release of upcoming fantasy film...


In [25]:
df_2018['overview'] = df_2018['description'].map(lambda x: x[0])
df_2018['tagline'] = df_2018['description'].map(lambda x: x[1])

<ipython-input-25-b66e1548fa5e>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2018['overview'] = df_2018['description'].map(lambda x: x[0])
<ipython-input-25-b66e1548fa5e>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2018['tagline'] = df_2018['description'].map(lambda x: x[1])


In [26]:
df_2018

,Title,id,description,overview,tagline
0,Insidious: The Last Key,406563,[Parapsychologist Elise Rainier and her team t...,Parapsychologist Elise Rainier and her team tr...,Fear comes home.
1,The Strange Ones,426258,[Mysterious events surround the travels of two...,Mysterious events surround the travels of two ...,
2,Stratton,348389,[A British Special Boat Service commando track...,A British Special Boat Service commando tracks...,The enemy has a weapon. So do we.
3,Sweet Country,468210,[In 1929 in the outback of the Northern Territ...,In 1929 in the outback of the Northern Territo...,Justice itself is put on trial
4,The Commuter,399035,"[A businessman, on his daily commute home, get...","A businessman, on his daily commute home, gets...",Lives are on the line
...,...,...,...,...,...
269,Holmes & Watson,426563,[Detective Sherlock Holmes and Dr. John Watson...,Detective Sherlock Holmes and Dr. John Watson ...,Holmies
270,Vice,82,[A case involving drug lords and murder in Sou...,A case involving drug lords and murder in Sout...,No Law. No Rules. No Order.
271,On the Basis of Sex,339380,[Young lawyer Ruth Bader Ginsburg teams with h...,Young lawyer Ruth Bader Ginsburg teams with he...,Her story made history
272,Destroyer,1040330,[Ahead of the release of upcoming fantasy film...,Ahead of the release of upcoming fantasy film ...,


In [27]:
df_2018['comb'] = df_2018['overview'] + df_2018['tagline']
df_2018['comb'] = df_2018['comb'].fillna('')

<ipython-input-27-94cf4953c996>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2018['comb'] = df_2018['overview'] + df_2018['tagline']
<ipython-input-27-94cf4953c996>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2018['comb'] = df_2018['comb'].fillna('')


In [28]:
df_2018 = df_2018.rename(columns={'Title':'title'})

In [29]:
df_2018 = df_2018.loc[:,['id','overview','tagline','title','comb']]

In [30]:
df_2018

,id,overview,tagline,title,comb
0,406563,Parapsychologist Elise Rainier and her team tr...,Fear comes home.,Insidious: The Last Key,Parapsychologist Elise Rainier and her team tr...
1,426258,Mysterious events surround the travels of two ...,,The Strange Ones,Mysterious events surround the travels of two ...
2,348389,A British Special Boat Service commando tracks...,The enemy has a weapon. So do we.,Stratton,A British Special Boat Service commando tracks...
3,468210,In 1929 in the outback of the Northern Territo...,Justice itself is put on trial,Sweet Country,In 1929 in the outback of the Northern Territo...
4,399035,"A businessman, on his daily commute home, gets...",Lives are on the line,The Commuter,"A businessman, on his daily commute home, gets..."
...,...,...,...,...,...
269,426563,Detective Sherlock Holmes and Dr. John Watson ...,Holmies,Holmes & Watson,Detective Sherlock Holmes and Dr. John Watson ...
270,82,A case involving drug lords and murder in Sout...,No Law. No Rules. No Order.,Vice,A case involving drug lords and murder in Sout...
271,339380,Young lawyer Ruth Bader Ginsburg teams with he...,Her story made history,On the Basis of Sex,Young lawyer Ruth Bader Ginsburg teams with he...
272,1040330,Ahead of the release of upcoming fantasy film ...,,Destroyer,Ahead of the release of upcoming fantasy film ...


In [31]:
df_2018.to_csv('desc2018.csv',index=False)

In [32]:
from google.colab import files
files.download("desc2018.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>